# Métodos directos - Casos especiales, pivotamiento e inversión

## Matrices de coeficientes simétricos y en banda

### Introducción

La modelización de problemas ingenieriles suele dar lugar a matrices de coeficientes poco pobladas, lo que significa que la mayoría de los elementos de la matriz son cero. Si todos los términos no nulos están agrupados en torno a la diagonal principal, se dice que la matriz está dividida en bandas. Un ejemplo de este tipo de matriz de coeficientes es:

$$
A=\left[\begin{array}{lllll}
\mathrm{X} & \mathrm{X} & 0 & 0 & 0 \\
\mathrm{X} & \mathrm{X} & \mathrm{X} & 0 & 0 \\
0 & \mathrm{X} & \mathrm{X} & \mathrm{X} & 0 \\
0 & 0 & \mathrm{X} & \mathrm{X} & \mathrm{X} \\
0 & 0 & 0 & \mathrm{X} & \mathrm{X}
\end{array}\right]
$$
donde las $X$ denotan los elementos no nulos que forman la banda poblada (algunos de estos elementos pueden ser cero). Todos los elementos que se encuentran fuera de esta banda son cero. Esta matriz con bandas tiene un ancho de banda de tres, porque hay como máximo tres elementos no nulos en cada fila (o columna). Este tipo de matriz se llama **tridiagonal**.

Si se descompone una matriz por bandas en la forma $\mathbf{A}=\mathbf{L U}$, tanto $\mathbf{L}$ como $\mathbf{U}$ conservan la
estructura de bandas de $\mathbf{A}$. Por ejemplo, si descomponemos la matriz mostrada anteriormente, obtendríamos

$$
\mathbf{L}=\left[\begin{array}{lllll}
\mathrm{X} & 0 & 0 & 0 & 0 \\
\mathrm{X} & \mathrm{X} & 0 & 0 & 0 \\
0 & \mathrm{X} & \mathrm{X} & 0 & 0 \\
0 & 0 & \mathrm{X} & \mathrm{X} & 0 \\
0 & 0 & 0 & \mathrm{X} & \mathrm{X}
\end{array}\right] \quad \mathbf{U}=\left[\begin{array}{ccccc}
\mathrm{X} & \mathrm{X} & 0 & 0 & 0 \\
0 & \mathrm{X} & \mathrm{X} & 0 & 0 \\
0 & 0 & \mathrm{X} & \mathrm{X} & 0 \\
0 & 0 & 0 & \mathrm{X} & \mathrm{X} \\
0 & 0 & 0 & 0 & \mathrm{X}
\end{array}\right]
$$

La estructura en bandas de una matriz de coeficientes puede aprovecharse para ahorrar tiempo de almacenamiento y cálculo. Si la matriz de coeficientes también es simétrica, aún se puede optimizar más este ahorro. En esta sección se muestra cómo los métodos de solución discutidos anteriormente pueden ser adaptados para matrices simétricas y en bandas.

### Matriz de coeficientes tridiagonal

Consideremos la solución de $\mathbf{Ax}=\mathbf{b}$ por la descomposición LU de Doolittle, donde $\mathbf{A}$ es una matriz tridiagonal $n \times n$ de la forma:

$$
\mathbf{A}=\left[\begin{array}{cccccc}
d_1 & e_1 & 0 & 0 & \ldots & 0 \\
c_1 & d_2 & e_2 & 0 & \ldots & 0 \\
0 & c_2 & d_3 & e_3 & \ldots & 0 \\
0 & 0 & c_3 & d_4 & \ldots & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & \ldots & 0 & c_{n-1} & d_n
\end{array}\right]
$$

Como se deduce de la notación anterior, almacenamos los elementos no nulos de $\mathbf{A}$ en los vectores

$$
\mathbf{c}=\left[\begin{array}{c}
c_1 \\
c_2 \\
\vdots \\
c_{n-1}
\end{array}\right] \quad \mathbf{d}=\left[\begin{array}{c}
d_1 \\
d_2 \\
\vdots \\
d_{n-1} \\
d_n
\end{array}\right] \quad \mathbf{e}=\left[\begin{array}{c}
e_1 \\
e_2 \\
\vdots \\
e_{n-1}
\end{array}\right]
$$

El ahorro en espacio de almacenamiento puede ser significativo. Por ejemplo, una matriz tridiagonal de $100 \times 100$ ($10000$ elementos) puede almacenarse en sólo $99 + 100 + 99 = 298$ espacios en la memoria, lo que representa una relación de compresión de aproximadamente $33:1$.

A continuación aplicamos la descomposición LU a la matriz de coeficientes. Reducimos la fila $k$ eliminando el coeficiente $c_{k−1}$ con la siguiente operación elemental:

$$
\text { fila } k \leftarrow \operatorname{fila} k-\left(c_{k-1} / d_{k-1}\right) \times \operatorname{fila}(k-1), \quad k=2,3, \ldots, n
$$

El cambio correspondiente en $d_k$ es

$$
d_k \leftarrow d_k-\left(c_{k-1} / d_{k-1}\right) e_{k-1}
$$
mientras que $e_k$ no se ve afectado. Para terminar con la descomposición de Doolittle de la forma $\left[ \mathbf{L} \ \mathbf{U} \right]$, almacenamos el multiplicador $\lambda = c_{k−1} / d_{k−1}$ en el lugar que antes ocupaba $c_{k-1}$:

$$
c_{k-1} \leftarrow c_{k-1} / d_{k-1}
$$

**Ejercicio 1 -** Escribe en Python una función que implemente la fase de eliminación del algoritmo de descomposición LU adaptado a matrices tridiagonales.

In [1]:
import numpy as np

In [20]:
def LU_tridiagonal_matrix(A):

    ''' 
    the input is a tridiagonal matrix A

    the output is a copy of the matrix A containing the coefficients updated by a lambda on the main diagonal
    and the corresponding lambdas on the lower diagonal.
    its a compact way of representing the LU to save memory storage.
    '''

    A_copy = np.copy(A)
    d = A.diagonal(0).copy()  # diag princ
    c = A.diagonal(-1).copy()  #diag inf
    e = A.diagonal(1).copy() #upper diag

    # print(d)
    # print(c)
    # print(e)

    n = len(d) 

    for k in range(1, n):
        lamb = c[k - 1] / d[k - 1]
        c[k - 1] = lamb
        d[k] = d[k] - lamb * e[k - 1]

    A_copy[np.arange(n), np.arange(n)] = d 
    A_copy[np.arange(1, n), np.arange(n-1)] = c    
    A_copy[np.arange(n-1), np.arange(1, n)] = e

    return A_copy

In [21]:
A = np.array([[4, 1, 0, 0],
            [2, 4, 1, 0],
            [0, 3, 4, 2],
            [0, 0, 1, 3]], dtype=float)

LU_tridiagonal_matrix(A)

array([[4.        , 1.        , 0.        , 0.        ],
       [0.5       , 3.5       , 1.        , 0.        ],
       [0.        , 0.85714286, 3.14285714, 2.        ],
       [0.        , 0.        , 0.31818182, 2.36363636]])

A continuación, se examina la fase de solución (es decir, la resolución de $\mathbf{Ly}=\mathbf{b}$, seguida de
$\mathbf{Ux}=\mathbf{y}$. Las ecuaciones $\mathbf{Ly}=\mathbf{b}$ pueden representarse mediante la matriz de coeficientes aumentada
 
$$
[\mathbf{L} \mid \mathbf{b}]=\left[\begin{array}{cccccc|c}
1 & 0 & 0 & 0 & \cdots & 0 & b_1 \\
c_1 & 1 & 0 & 0 & \cdots & 0 & b_2 \\
0 & c_2 & 1 & 0 & \cdots & 0 & b_3 \\
0 & 0 & c_3 & 1 & \cdots & 0 & b_4 \\
\vdots & \vdots & \vdots & \vdots & \cdots & \vdots & \vdots \\
0 & 0 & \cdots & 0 & c_{n-1} & 1 & b_n
\end{array}\right]
$$
Nótese que el contenido original de $\mathbf{c}$ fue eliminado y sustituido por los multiplicadores durante la descomposición. El algoritmo de solución de $\mathbf{y}$ por sustitución hacia adelante es

```
y[0] <- b[0]
Para k desde 1 hasta n-1 hacer:
    y[k] <- b[k] - c[k-1] * y[k-1]
Fin Para
```
La matriz de coeficientes aumentada que representa $\mathbf{Ux}=\mathbf{y}$ es

$$
[\mathbf{U} \mid \mathbf{y}]=\left[\begin{array}{cccccc|c}
d_1 & e_1 & 0 & \cdots & 0 & 0 & y_1 \\
0 & d_2 & e_2 & \cdots & 0 & 0 & y_2 \\
0 & 0 & d_3 & \cdots & 0 & 0 & y_3 \\
\vdots & \vdots & \vdots & & \vdots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & d_{n-1} & e_{n-1} & y_{n-1} \\
0 & 0 & 0 & \cdots & 0 & d_n & y_n
\end{array}\right]
$$

Observa de nuevo que el contenido de $\mathbf{d}$ se ha modificado respecto a los valores originales durante la fase de descomposición (sin embargo, $\mathbf{e}$ no se ha modificado). La solución para $\mathbf{x}$ se obtiene mediante la sustitución hacia atrás, utilizando el algoritmo siguiente:

```
x[n-1] <- y[n-1] / d[n-1]
Para k desde n-2 hasta 0 en pasos decrecientes hacer:
    x[k] <- (y[k] - e[k] * x[k+1]) / d[k]
Fin Para
```

**Ejercicio 2 -** Combina el código del ejercicio 1 junto con el algoritmo de sustitución explicado anteriormente para programar una función que resuelva un sistema lineal $\mathbf{AX}=\mathbf{b}$ tridiagonal por el método LU adaptado visto anteriormente.

In [22]:
def solve_LU_tridiagonal_matrix(LU, b):
    '''
    the input is a matriz decomposition LU, and the solution vector b

    returns vector x with the solutions
    '''
    d = LU.diagonal(0).copy()  # diag princ
    c = LU.diagonal(-1).copy()  #diag inf
    e = LU.diagonal(1).copy() #upper diag

    y = np.zeros(d.shape)
    n = len(d)

    y[0] = b[0]
    for k in range(1, n):
        y[k] = b[k] - c[k-1] * y[k-1]

    x = np.zeros(d.shape)
    x[-1] = y[-1] / d[-1]
    for k in range(n-2, -1, -1):
        x[k] = (y[k] - e[k] * x[k+1]) / d[k]

    return x

In [23]:
A = np.array([[4, 1, 0, 0],
            [2, 4, 1, 0],
            [0, 3, 4, 2],
            [0, 0, 1, 3]], dtype=float)

b = np.array([15, 8, 22, 10], dtype=float)

LU = LU_tridiagonal_matrix(A)

print(f' el resultado por numpy es = \n {np.linalg.solve(A, b)}')

solve_LU_tridiagonal_matrix(LU, b)

 el resultado por numpy es = 
 [ 4.14423077 -1.57692308  6.01923077  1.32692308]


array([ 4.14423077, -1.57692308,  6.01923077,  1.32692308])

### Matriz de coeficientes simétricos


A menudo, las matrices de coeficientes que surgen en los problemas de ingeniería son simétricas y con bandas. Por lo tanto, merece la pena detenerse en las propiedades particulares de dichas matrices para utilizarlas en la construcción de algoritmos eficientes.
Si la matriz $\mathbf{A}$ es simétrica, la descomposición LU puede escribirse de la forma

$$
\mathbf{A}=\mathbf{L U}=\mathbf{L D L}^T
$$
donde $\mathbf{D}$ es una matriz diagonal. Un ejemplo es la descomposición de Choleski $\mathbf{A}=\mathbf{L L}^T$  que se estudió anteriormente (en ese caso $\mathbf{D}=\mathbf{I}$).
Para la descomposición de Doolittle tenemos
$$
\mathbf{U}=\mathbf{D L}^T=\left[\begin{array}{ccccc}
D_1 & 0 & 0 & \cdots & 0 \\
0 & D_2 & 0 & \cdots & 0 \\
0 & 0 & D_3 & \cdots & 0 \\
\vdots & \vdots & \vdots & \cdots & \vdots \\
0 & 0 & 0 & \cdots & D_n
\end{array}\right]\left[\begin{array}{ccccc}
1 & L_{21} & L_{31} & \cdots & L_{n 1} \\
0 & 1 & L_{32} & \cdots & L_{n 2} \\
0 & 0 & 1 & \cdots & L_{n 3} \\
\vdots & \vdots & \vdots & \cdots & \vdots \\
0 & 0 & 0 & \cdots & 1
\end{array}\right]
$$
que da lugar a:
$$
\mathbf{U}=\left[\begin{array}{ccccc}
D_1 & D_1 L_{21} & D_1 L_{31} & \cdots & D_1 L_{n 1} \\
0 & D_2 & D_2 L_{32} & \cdots & D_2 L_{n 2} \\
0 & 0 & D_3 & \cdots & D_3 L_{3 n} \\
\vdots & \vdots & \vdots & \cdots & \vdots \\
0 & 0 & 0 & \cdots & D_n
\end{array}\right]
$$
Ahora vemos que durante la descomposición de una matriz simétrica sólo hay que almacenar $\mathbf{U}$, porque $\mathbf{D}$ y $\mathbf{L}$ se pueden recuperar fácilmente a partir de $\mathbf{U}$. Así, la eliminación de Gauss, que da lugar a una matriz triangular superior de la forma anterior, es suficiente para descomponer una matriz simétrica.

**Ejercicio 3 -** Como resultado de la eliminación de Gauss, una matriz simétrica $\mathbf{A}$ se transformó en la forma triangular superior

$$
\mathbf{U}=\left[\begin{array}{rrrr}
4 & -2 & 1 & 0 \\
0 & 3 & -3 / 2 & 1 \\
0 & 0 & 3 & -3 / 2 \\
0 & 0 & 0 & 35 / 12
\end{array}\right]
$$

Determina la matriz original $\mathbf{A}$.


In [2]:
import numpy as np

In [24]:
U = np.array([[4, -2, 1, 0],
              [0, 3, -3/2, 1],
              [0, 0, 3, -3/2],
              [0, 0, 0, 35/12]], dtype=float)


d = U.diagonal(0).copy()
U_copy = np.copy(U)

for i in range(len(U) - 2):
    U_copy[i, i+1:] = U[i, i+1:] / d[i]

print(U)
print()
print(U_copy)

[[ 4.         -2.          1.          0.        ]
 [ 0.          3.         -1.5         1.        ]
 [ 0.          0.          3.         -1.5       ]
 [ 0.          0.          0.          2.91666667]]

[[ 4.         -0.5         0.25        0.        ]
 [ 0.          3.         -0.5         0.33333333]
 [ 0.          0.          3.         -1.5       ]
 [ 0.          0.          0.          2.91666667]]


In [15]:
L = U_copy.T
n = len(d)
L[np.arange(n), np.arange(n)] = np.ones(d.shape)
L

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [-0.5       ,  1.        ,  0.        ,  0.        ],
       [ 0.25      , -0.5       ,  1.        ,  0.        ],
       [ 0.        ,  0.33333333, -1.5       ,  1.        ]])

In [16]:
A = L@U
A

array([[ 4. , -2. ,  1. ,  0. ],
       [-2. ,  4. , -2. ,  1. ],
       [ 1. , -2. ,  4. , -2. ],
       [ 0. ,  1. , -5. ,  5.5]])

**Ejercicio 4 -** Determinar L y D que resultan de la descomposición de Doolittle $\mathbf{A} = \mathbf{LDL}^T$ de la matriz simétrica
$$
A=\left[\begin{array}{rrr}
3 & -3 & 3 \\
-3 & 5 & 1 \\
3 & 1 & 10
\end{array}\right]
$$

In [105]:
# forma programada de gpt

def LDL_decomposition(A):
    """
    Realiza la descomposición LDL^T de una matriz simétrica A.
    Retorna las matrices L y D.
    """
    n = A.shape[0]
    L = np.eye(n)  # Inicializamos L como la identidad
    D = np.zeros((n, n))  # Inicializamos D como una matriz diagonal

    for j in range(n):
        # Calcular D[j, j]
        D[j, j] = A[j, j] - sum(L[j, k]**2 * D[k, k] for k in range(j))
        
        for i in range(j + 1, n):
            # Calcular L[i, j]
            L[i, j] = (A[i, j] - sum(L[i, k] * L[j, k] * D[k, k] for k in range(j))) / D[j, j]
    
    return L, D


# Ejemplo de uso
A = np.array([
    [4, -2, 1, 0],
    [-2, 4, -2, 1],
    [1, -2, 4, -2],
    [0, 1, -2, 4]
], dtype=float)

L, D = LDL_decomposition(A)
L, D


(array([[ 1.        ,  0.        ,  0.        ,  0.        ],
        [-0.5       ,  1.        ,  0.        ,  0.        ],
        [ 0.25      , -0.5       ,  1.        ,  0.        ],
        [ 0.        ,  0.33333333, -0.5       ,  1.        ]]),
 array([[4.        , 0.        , 0.        , 0.        ],
        [0.        , 3.        , 0.        , 0.        ],
        [0.        , 0.        , 3.        , 0.        ],
        [0.        , 0.        , 0.        , 2.91666667]]))

**Ejercicio 5 -** Utiliza las funciones programadas en los ejercicios 1 y 2 para resolver el sistema tridiagonal definido por las siguientes matrices:
$$
\mathbf{A}=\left[\begin{array}{rrrrr}
2 & -1 & 0 & 0 & 0 \\
-1 & 2 & -1 & 0 & 0 \\
0 & -1 & 2 & -1 & 0 \\
0 & 0 & -1 & 2 & -1 \\
0 & 0 & 0 & -1 & 2
\end{array}\right] \quad \mathbf{b}=\left[\begin{array}{r}
5 \\
-5 \\
4 \\
-5 \\
5
\end{array}\right]
$$

In [102]:
A = np.array([
    [2, -1,  0,  0,  0],
    [-1, 2, -1,  0,  0],
    [0, -1,  2, -1,  0],
    [0,  0, -1,  2, -1],
    [0,  0,  0, -1,  2]
], dtype=float)

b = np.array([5, -5, 4, -5, 5], dtype=float)

In [104]:
LU = LU_tridiagonal_matrix(A)

solution_5 = solve_LU_tridiagonal_matrix(LU, b)
solution_5

array([ 2., -1.,  1., -1.,  2.])

## Pivotamiento

### Introducción
A veces, el orden en que se presentan las ecuaciones al algoritmo de solución tiene un efecto determinante en los resultados. Por ejemplo, consideremos estas ecuaciones:

$$
\begin{array}{r}
2 x_1-x_2=1 \\
-x_1+2 x_2-x_3=0 \\
-x_2+x_3=0
\end{array}
$$

La correspondiente matriz de coeficientes aumentada es
$$
[\mathbf{A} \mid \mathbf{b}]=\left[\begin{array}{rrr|r}
2 & -1 & 0 & 1 \\
-1 & 2 & -1 & 0 \\
0 & -1 & 1 & 0
\end{array}\right]
$$
 
Las ecuaciones anteriores están en el "orden correcto" en el sentido de que no tendríamos problemas para obtener la solución correcta $x_1 = x_2 = x_3 = 1$ por eliminación de Gauss o descomposición LU. Supongamos ahora que intercambiamos las ecuaciones primera y tercera, de modo que la matriz de coeficientes aumentada se convierte en

$$
[\mathbf{A} \mid \mathbf{b}]=\left[\begin{array}{rrr|r}
0 & -1 & 1 & 0 \\
-1 & 2 & -1 & 0 \\
2 & -1 & 0 & 1
\end{array}\right]
$$

Como no hemos cambiado las ecuaciones (sólo se ha alterado su orden), la solución sigue siendo $x_1 = x_2 = x_3 = 1$. Sin embargo, la eliminación de Gauss falla inmediatamente debido a la presencia del elemento pivote cero (el elemento $A_{11}$).

Este ejemplo demuestra que a veces es esencial reordenar las ecuaciones durante la fase de eliminación. La reordenación, o pivotamiento de filas, también es necesaria si el elemento pivote no es cero, pero es muy pequeño en comparación con otros elementos de la fila pivote, como demuestra el siguiente conjunto de ecuaciones:

$$
[\mathbf{A} \mid \mathbf{b}]=\left[\begin{array}{rrr|r}
\varepsilon & -1 & 1 & 0 \\
-1 & 2 & -1 & 0 \\
2 & -1 & 0 & 1
\end{array}\right]
$$

Estas ecuaciones son las mismas que las ecuaciones del ejemplo anterior, excepto que se usa un número pequeño $\varepsilon$ en lugar del cero anterior. Por lo tanto, si $\varepsilon \to 0$, la solución de este sistema de ecuaciones debería ser igual a la del anterior. Después de la primera fase de eliminación de Gauss, la matriz de coeficientes aumentada se convierte en
$$
\left[\begin{array}{l|l}
\mathbf{A}^{\prime} & \mathbf{b}^{\prime}
\end{array}\right]=\left[\begin{array}{ccc|c}
\varepsilon & -1 & 1 & 0 \\
0 & 2-1 / \varepsilon & -1+1 / \varepsilon & 0 \\
0 & -1+2 / \varepsilon & -2 / \varepsilon & 1
\end{array}\right]
$$

Como el ordenador trabaja con una longitud de representación fija, todos los números se redondean a un número finito de cifras significativas. Si $\varepsilon$ es muy pequeño, entonces $1/ \varepsilon$ es muy grande, y un elemento como $2 - 1/ \varepsilon $ se redondeará a $- 1/ \varepsilon$. Por lo tanto, para un $\varepsilon$ suficientemente pequeño, el sistema de ecuaciones anterior es almacenado realmente como:

$$
\left[\begin{array}{l|l}
\mathbf{A}^{\prime} & \mathbf{b}^{\prime}
\end{array}\right]=\left[\begin{array}{ccc|c}
\varepsilon & -1 & 1 & 0 \\
0 & -1 / \varepsilon & 1 / \varepsilon & 0 \\
0 & 2 / \varepsilon & -2 / \varepsilon & 1
\end{array}\right]
$$

Como las ecuaciones segunda y tercera se contradicen obviamente, el proceso de resolución falla. Este problema no se produciría si se intercambiaran la primera y la segunda, o la primera y la tercera ecuación en el sistema original antes de la eliminación.

El último ejemplo ilustra un caso extremo en el que un $\varepsilon$ pequeño causa que los errores de redondeo desencadenen un fracaso total. Si hiciéramos $\varepsilon$ algo más grande para que la solución no "explotara", los errores de redondeo podrían seguir siendo lo suficientemente grandes como para que la solución no fuera fiable. Esta dificultad podría evitarse pivotando.

### Diagonal dominante 

Se dice que una matriz $\mathbf{A}$ de dimensión $n \times n$ es *diagonal dominante* si cada elemento de la diagonal es mayor que la suma de los demás elementos de la misma fila (en valores absolutos). Por tanto, la dominancia diagonal requiere que
$$
\left|A_{i i}\right|>\sum_{\substack{j=1 \\ j \neq i}}^n\left|A_{i j}\right|(i=1,2, \ldots, n)
$$
Por ejemplo, la matriz

$$
\left[\begin{array}{rrr}
-2 & 4 & -1 \\
1 & -1 & 3 \\
4 & -2 & 1
\end{array}\right]
$$

no es diagonalmente dominante. Sin embargo, si reordenamos las filas de la siguiente manera

$$
\left[\begin{array}{rrr}
4 & -2 & 1 \\
-2 & 4 & -1 \\
1 & -1 & 3
\end{array}\right]
$$
entonces tenemos dominio diagonal.

Se puede demostrar que si la matriz de coeficientes de las ecuaciones $\mathbf{Ax} = \mathbf{b}$ es diagonalmente dominante, entonces la resolución del sistema no será más eficiente con pivotamiento; es decir, las ecuaciones ya están dispuestas en el orden óptimo. De ello se deduce que la estrategia de pivotamiento debe consistir en reordenar las ecuaciones de forma que la matriz de coeficientes se acerque lo más posible a ser diagonal dominante. Este es el principio en el que se basa el pivotamiento, que se discute a continuación.

### Eliminación de Gauss con pivotamiento

Consideremos la solución de $\mathbf{Ax} = \mathbf{b}$ por eliminación de Gauss con pivotamiento de filas. Recordemos que el pivote tiene como objetivo mejorar la dominancia diagonal de la matriz de coeficientes (es decir, hacer que el elemento pivote sea lo más grande posible en comparación con otros elementos de la fila pivote). Esta comparación es más sencilla si manejamos una matriz $s$ con los elementos

$$
s_i=\max _j\left|A_{i j}\right|, \quad i=1,2, \ldots, n
$$

Así $s_i$, llamado factor de escala de la fila $i$, contiene el valor absoluto del mayor elemento de la fila $i$ de $\mathbf{A}$. El vector $s$ puede obtenerse trivialmente como:

```
Para i desde 0 hasta n-1 hacer:
    s[i] = máximo valor absoluto de los elementos en la fila i de la matriz a
Fin Para
```

El tamaño relativo de un elemento $A_{ij}$ (es decir, en relación con el elemento más grande del la fila $i$) se define como la relación:

$$
r_{i j}=\frac{\left|A_{i j}\right|}{s_i}
$$


Supongamos que la fase de eliminación ha llegado a la etapa en la que la fila $k$ se ha convertido en la fila pivote. La matriz de coeficientes aumentada en este punto tenrá la siguiente forma:
 
$$
\left[\begin{array}{cccccc|c}
A_{11} & A_{12} & A_{13} & A_{14} & \cdots & A_{1 n} & b_1 \\
0 & A_{22} & A_{23} & A_{24} & \cdots & A_{2 n} & b_2 \\
0 & 0 & A_{33} & A_{34} & \cdots & A_{3 n} & b_3 \\
\vdots & \vdots & \vdots & \vdots & \cdots & \vdots & \vdots \\
\hline 0 & \cdots & 0 & A_{k k} & \cdots & A_{k n} & b_k \\
\vdots & \cdots & \vdots & \vdots & \cdots & \vdots & \vdots \\
0 & \cdots & 0 & A_{n k} & \cdots & A_{n n} & b_n
\end{array}\right]
$$

No aceptamos automáticamente $A_{kk}$ como el siguiente elemento pivote, sino que buscamos en la $k$-ésima columna por debajo de $A_{kk}$ un pivote "mejor". La mejor elección es el elemento $A_{pk}$ que tenga el mayor tamaño relativo; es decir, elegimos $p$ de forma que:

$$
r_{p k}=\max _j\left(r_{j k}\right), \quad j \geq k
$$

Si encontramos un elemento de este tipo, entonces intercambiamos las filas $k$ y $p$, y procedemos a la pasada de eliminación como es habitual. Obsérvese que el correspondiente intercambio de filas debe realizarse también en el vector del factor de escala $s$. El algoritmo que realiza todo esto es el siguiente:

```
Para k desde 0 hasta n-2 hacer:
    # Encontrar la fila con el elemento de mayor tamaño relativo
    p = índice del máximo valor absoluto de a[k:n,k] dividido por s[k:n] + k

    # Si este elemento es muy pequeño, la matriz es singular
    Si abs(a[p,k]) < tol entonces
        Mostrar error: 'Matriz singular'

    # Verificar si las filas k y p deben intercambiarse
    Si p != k entonces
        # Intercambiar filas si es necesario
        Intercambiar filas k y p de b
        Intercambiar filas k y p de s 
        Intercambiar filas k y p de a

    # Continuar con la eliminación...
Fin Para
```

A la hora de implementar en python, puedes usar el método ```argmax(v)``` para obtener el índice del mayor elemento de un cierto vector ```v```. Por último, a continuación se incluye el pseudocódigo de una función llamada ```swap_rows()``` para intercambiar filas:

```
Función swap_rows(v, i, j)
    Descripción: Intercambia las filas i y j de un vector o matriz [v].

    Si el número de dimensiones de v es 1 entonces
        Intercambiar los elementos v[i] y v[j]
    Sino
        Intercambiar las filas i y j de la matriz v
    Fin Si
Fin Función
```

In [1]:
import numpy as np
A = np.array([
    [2, -2, 6],
    [-2, 4, 3],
    [-1, 8, 4]
], dtype=float)

b = np.array([16, 0, -1], dtype=float)

i, j = 1, 2
A

A[[i, j]] = A[[j, i]]
b[[i, j]] = b[[j, i]]

s = np.amax(A, axis=1)
s

array([6., 8., 4.])

**Ejercicio 6 -** Programa una función llamada ```gauss_pivot``` que implemente el método de Gauss con pivotamiento. Reutiliza el código de secciones anteriores. Ten en cuenta que la fase de sustitución es idéntica al método de Gauss sin pivotamiento.

In [47]:
def swap_rows(A, i , j):
    '''intercambia la fija j por la i 
    en la matriz de coeficientes A
    de un sistema lienal
    return una nueva matriz A con las filas intercambiadad
    pero con la matriz A original sin modificar'''

    A[[i, j]] = A[[j, i]]

    return A

def row_lineal_comb(A, i, j, k):

    A[j] = A[j] - A[i] * k

    return A

def backward_sustitusion(U, c):
    '''
    the input is an upper triangular matrix U which dimension is nxn
    and c vector which contains the results, variable dimension
    the output is vector x, which initialy contained the unkwons, with the solution
    '''
    A_copy = np.copy(U)
    b_copy = np.copy(c)
    
    n = len(c)

    x = np.zeros((n,))


    for i in range(n-1, -1, -1):
        total_sum = 0
        for j in range(i+1, n):
            total_sum += A_copy[i, j] * x[j]

        x[i] = (b_copy[i] - total_sum) / A_copy[i, i]


    return x

In [48]:
def gauss_pivot(A, b, tol=1.0e-12, func1 = swap_rows, func2= row_lineal_comb, func3= backward_sustitusion):

    n = len(A)
    
    s = np.amax(np.abs(A), axis= 1)

    # print('s = ', s)

    for k in range(n-1):
        #print(f'k = {k}')

        r = [np.abs(A[i, k]) / s[i] for i in range(k, len(s))]
        #print(f'r = {r}')
        p = np.argmax(r) + k
        #print(f'p = {p}')


        if p != k:
            A = func1(A, k, p)
            #print(f'A swap rows {k, p} = \n{A}')
            b = func1(b, k, p)
            #print(f'b swap rows {k, p} = {b}')
            s = func1(s, k, p)
            #print(f's swap rows {k, p} = {s}')

            for i in range(k+1, n):
                #print(f'i = {i}')
                c = A[i, k] / A[k, k]
                #print(f'c = {c}')
                A = func2(A, k , i, c)
                #print(f'A = {A}')
                b[i] -= c * b[k]
                #print(f'b = {b}')
    sol = func3(A, b)

    return sol

In [36]:
#ejemplo uso

A = np.array([
    [2, -2, 6],
    [-2, 4, 3],
    [-1, 8, 4]
], dtype=float)

b = np.array([16, 0, -1], dtype=float)

gauss_pivot(A, b)


array([ 1., -1.,  2.])

In [16]:
n = 5

seq = np.arange(n)
seq

array([0, 1, 2, 3, 4])

### Método LU con pivotamiento

El algoritmo de eliminación de Gauss puede extenderse a la descomposición de Doolittle con pequeños cambios. El más importante de estos cambios es mantener un registro de los intercambios de filas durante la fase de descomposición. Para ello, se suele manejar un vector adicional que lleve el registro de estos cambios. Si inicialmente definimos un array ```seq = [0, 1, 2, ... ]```, cada vez que se cambian dos filas, el intercambio correspondiente también se realizará en ```seq```. Así, ```seq``` mostrará el orden en que se han reordenado las filas originales. Esta información se transmite a la fase de solución, que reordena los elementos del vector constante en el mismo orden antes de proceder a las sustituciones hacia delante y hacia atrás.

**Ejercicio 7 -** Programa una función ```pivotLU``` que implemente el método LU con pivotamiento. Ten en cuenta las consideraciones anteriores. Reutiliza el código programado en secciones anteriores.

In [49]:
def forward_backward_sustitution(L, U, b, func= backward_sustitusion):
    '''
    the input is a lower trriangular matrix L,
    an upper triangular matrix U,
    yhe solution vector b
    and the function which will be a normal backward sustitusion by default,
    the function of this parameter will be dealing with the second part of fthe solving
    

    reminder: we are searching for x, not y
    '''

    assert len(b) >= 1, 'this method isnt valid for solving multiply ecuation systems'

    #forward sustitutuon
    # Ux = y -> Solve Ly = b

    L_copy = np.copy(L)
    b_copy = np.copy(b)
    y = np.zeros(b.shape)

    for i in range(len(L)):
        total_sum = 0
        for j in range(i, -1, -1):
            total_sum += L_copy[i, j] * y[j]
            # print(f'total sum = {total_sum}')

        y[i] = (b_copy[i] - total_sum) / L_copy[i, i]
        # print(f'y = {y}')

    #backward sustitution
    # Ux = y
    solution = func(U, y)

    return solution


In [ ]:
def gauss_LU_pivot(A, b = None, tol=1.0e-12, func1 = swap_rows, func2= row_lineal_comb, func3= forward_backward_sustitution):

    n = len(A)
    
    s = np.amax(np.abs(A), axis= 1)

    seq = np.arange(n)

    L = np.eye((n))

    for k in range(n-1):
        r = [np.abs(A[i, k]) / s[i] for i in range(k, len(s))]
        p = np.argmax(r) + k

        if p != k:
            A = func1(A, k, p)
            seq = func1(seq, k, p)
            s = func1(s, k, p)

            for i in range(k+1, n):
                c = A[i, k] / A[k, k]
                L[i, k] = c
                A = func2(A, k , i, c)
                
    print(L)
    # print(A)
    U = A   
    
    if b is not None:  # Si b no es False, resolvemos el sistema
        b_reordered = b[seq]
        # print(b_reordered)
        sol = func3(L, U, b_reordered)    
        return sol  
    
    return L, U


A = np.array([
    [2, -2, 6],
    [-2, 4, 3],
    [-1, 8, 4]
], dtype=float)

b = np.array([16, 0, -1], dtype=float)

print(gauss_LU_pivot(A, b))
print()
L, U = gauss_LU_pivot(A)
print("L:\n", L)
print("U:\n", U)

[[ 1.          0.          0.        ]
 [-1.          1.          0.        ]
 [ 0.5         0.33333333  1.        ]]
[ 1. -1.  2.]

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
L:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
U:
 [[-2.          4.          3.        ]
 [ 0.          6.          2.5       ]
 [ 0.          0.          8.16666667]]


### Cuándo pivotar

El pivotamiento tiene dos inconvenientes. Uno es el aumento del coste computacional; el otro es la posible destrucción de la simetría o estructura por bandas de la matriz de coeficientes. Esto último es especialmente preocupante en problemas ingenieriles, donde las matrices de coeficientes son frecuentemente por bandas y simétricas, una propiedad que se utiliza en el proceso de resolución, como se ha visto anteriormente. Por suerte, estas matrices suelen ser también diagonal dominantes, por lo que no se beneficiarían del pivotamiento de todos modos.

No hay reglas infalibles para determinar cuándo debe utilizarse el pivotamiento. La experiencia indica que el pivotamiento es probablemente contraproducente si la matriz de coeficientes es por bandas. De forma análoga, las matrices definidas positivas y las simétricas rara vez se benefician del pivotamiento. Recuerda que el pivotamiento pretende solucionar problemas derivados de los errores de redondeo: muchas veces es mejor aumentar simplemente la precisión de cálculo en casos problemáticos.

**Ejercicio 8 -** Opera a mano el sistema de ecuaciones definido por las siguientes matrices, siguiendo el método de Gauss con pivotamiento. Recuerda comenzar definiendo un vector $s$ donde vayas almacenando el mayor elemento de cada fila en las distintas etapas del algoritmo. Comprueba que el vector solución al que llegas es el $[1, -1, 2]$.
$$
\mathbf{A}=\left[\begin{array}{rrr}
2 & -2 & 6 \\
-2 & 4 & 3 \\
-1 & 8 & 4
\end{array}\right] \quad \mathbf{b}=\left[\begin{array}{r}
16 \\
0 \\
-1
\end{array}\right]
$$

## Inversión de matrices

El cálculo de la inversa de una matriz y la resolución de sistemas de ecuaciones son problemas relacionados. La forma más económica de invertir una matriz $\mathbf{A}$ de $n \times n$ es resolver el sistema de ecuaciones:

$$
\mathbf{AX} = \mathbf{I}
$$

donde $\mathbf{I}$ es la matriz identidad. La solución $\mathbf{X}$, también de tamaño $n \times n$, será la inversa de $\mathbf{A}$. La prueba es sencilla: tras premultiplicar ambos lados de la ecuación anterior por $\mathbf{A}^{-1}$ tenemos $\mathbf{A}^{-1} \mathbf{AX} = \mathbf{A}^{-1} \mathbf{I}$, lo que se reduce a $\mathbf{X} = \mathbf{A}^{-1}$.

La inversión de matrices grandes debe evitarse siempre que sea posible debido a su alto coste. Como se aprecia de la ecuación anterior, la inversión de $\mathbf{A}$ es equivalente a resolver $\mathbf{Ax_i} = \mathbf{b_i}$ con $i = 1, 2, \dots , n,$ donde $\mathbf{b_i}$ es la $i$-ésima columna de $\mathbf{I}$. Suponiendo que se utilizase la descomposición LU, la fase de resolución (sustitución hacia delante y hacia atrás) debería repetirse $n$ veces, una por cada $\mathbf{b_i}$. Dado que el algoritmo es $\mathcal{O}\left(n^3\right)$ para la fase de descomposición y $\mathcal{O}\left(n^2\right)$ para cada vector en la fase de resolución, el coste de la inversión es considerablemente superior que el de la solución de un único sistema.

La inversión de matrices tiene otro grave inconveniente: una matriz con bandas pierde su estructura durante la inversión. En otras palabras, si $\mathbf{A}$ tiene bandas o es dispersa, entonces $\mathbf{A}^{-1}$ está completamente poblada.




**Ejercicio 9 -** Escribe una función que invierta una matriz utilizando la descomposición LU con pivote. Prueba la función invirtiendo la siguiente matriz:

$$
\mathbf{A}=\left[\begin{array}{rrr}
0.6 & -0.4 & 1.0 \\
-0.3 & 0.2 & 0.5 \\
0.6 & -1.0 & 0.5
\end{array}\right]
$$


In [3]:
import numpy as np

In [24]:
def row_lineal_comb(A, i, j, k):

    ''' i es la posicion de la linea a la que multiplicas por k
    j es el indice de la linea que estamos modificando
    k es el valor por el cual multiplicamos a A[i]

    devuelve uan copia de la matriz 
    '''

    A_copy = np.copy(A)
    ai = np.copy(A[i])

    A_copy[j] = A_copy[j] + ai * k

    return A_copy


In [25]:
#gpt version, pendiente de checkearlo

def solve_backward_substitution(U, c):
    '''
    U: upper triangular matrix
    c: column vector or matrix (result from forward substitution)
    '''
    # Si c es un vector, convertirlo en 2D para unificar
    if len(c.shape) == 1:
        c = c.reshape(-1, 1)

    n = c.shape[0]
    X = np.zeros_like(c)  # Matriz para soluciones

    for k in range(c.shape[1]):  # Resolver columna por columna
        b = c[:, k]  # Tomar cada columna como un vector
        x = np.zeros(n)
        for i in range(n - 1, -1, -1):
            total_sum = sum(U[i, j] * x[j] for j in range(i + 1, n))
            x[i] = (b[i] - total_sum) / U[i, i]
        X[:, k] = x

    return X


def solve_forward_substitution(L, B):
    '''
    L: Matriz triangular inferior
    B: Matriz de soluciones o vector (si es un vector, se convierte en matriz de una sola columna)
    Retorna la matriz Y que satisface Ly = B
    '''
    # Convertir B en matriz si es un vector
    if len(B.shape) == 1:
        B = B.reshape(-1, 1)

    # Inicializamos Y
    Y = np.zeros_like(B)

    # Resolución hacia adelante para cada columna
    for k in range(B.shape[1]):
        b = B[:, k]  # Tomar la columna k como vector
        y = np.zeros_like(b)
        for i in range(len(L)):
            total_sum = sum(L[i, j] * y[j] for j in range(i))
            y[i] = (b[i] - total_sum) / L[i, i]
        Y[:, k] = y  # Guardar la solución de la columna

    return Y

def solve_forward_backward_substitution(L, U, B, func1 = solve_forward_substitution,func2=solve_backward_substitution):
    '''
    L: Matriz triangular inferior
    U: Matriz triangular superior
    B: Matriz de soluciones o vector
    func: Función para la sustitución hacia atrás (por defecto backward_sustitution)
    Retorna la matriz X que satisface LUx = B
    '''
    # Resolver hacia adelante: Ly = B
    Y = func1(L, B)

    # Resolver hacia atrás: Ux = Y
    X = func2(U, Y)

    return X



In [ ]:
def calculate_inversa(A,  func1 = gauss_LU_pivot, func2 = solve_forward_backward_substitution):
    n = len(A)
    I  = np.eye(n)

    L, U = func1(A)

    print(U)

    resul = func2(L, U, I)

    return resul


In [27]:
A = np.array([[0.6, -0.4, 1.0],
              [-0.3, 0.2, 0.5],
              [0.6, -1.0, 0.5]])

calculate_inversa(A)

TypeError: gauss_LU_pivot() missing 1 required positional argument: 'b'

In [14]:
np.linalg.inv(A)

array([[ 1.66666667, -2.22222222, -1.11111111],
       [ 1.25      , -0.83333333, -1.66666667],
       [ 0.5       ,  1.        ,  0.        ]])